## 7.3 베이스라인 모델

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import pandas as pd

data_path = '/kaggle/input/cat-in-the-dat/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

### 7.3.1 피처 엔지니어링

* 데이터 합치기

In [ ]:
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1)
all_data

* 원-핫 인코딩

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
all_data_encoded = encoder.fit_transform(all_data)

* 데이터 나누기

In [ ]:
num_train = len(train)

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data_encoded[:num_train]
X_test = all_data_encoded[num_train:]

y = train['target']

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련데이터, 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=0.1, stratify=y, random_state=10)

### 7.3.2 모델 훈련

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train, y_train)

* max_iter: 모델의 회귀 계수를 업데이트하는 반복 횟수, 모델 훈련 시 회귀 계수를 업데이트하면서 훈련하는데, 이때 업데이트를 몇 번 할지를 정한다.

### 7.3.3 모델성능 검증

In [ ]:
logistic_model.predict_proba(X_valid)

In [ ]:
logistic_model.predict(X_valid)

In [ ]:
y_valid_preds = logistic_model.predict_proba(X_valid)[:, 1]

In [ ]:
from sklearn.metrics import roc_auc_score

# 검증 데이터 ROC AUC
roc_auc = roc_auc_score(y_valid, y_valid_preds)
print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

### 7.3.4 예측 및 결과 제출

In [ ]:
y_preds = logistic_model.predict_proba(X_test)[:, 1]

In [ ]:
# 제출 파일 생성
submission['target'] = y_preds
submission.to_csv('submission.csv')